In [1]:
import overpy
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np
import time

def geocode_address(address):
    geolocator = Nominatim(user_agent="your_app_name")
    location = geolocator.geocode(address)
    time.sleep(1)
    return (location.latitude, location.longitude)



def get_nearby_amenities(address):
    try:
        coordinates = geocode_address(address)

        overpass_query = f"""
            node(around:500, {coordinates[0]}, {coordinates[1]})
            ["amenity"];
            out;
        """

        api = overpy.Overpass()
        result = api.query(overpass_query)

        amenities = [node.tags.get('amenity', 'N/A') for node in result.nodes]
        time.sleep(1)
        return ", ".join(amenities)
    
    except Exception as e:
        return str(e)

def process_excel(input_file, output_file):
    df = pd.read_excel(input_file)

    df['Amenities'] = df['Address'].apply(get_nearby_amenities)

    df.to_csv(output_file, index=False)

# Example usage
input_excel_file = "final_input_addresses.xlsx"
output_csv_file = "final_output_amenities.csv"
process_excel(input_excel_file, output_csv_file)


In [2]:
import re
# Create data frame
csv = pd.read_csv("final_output_amenities.csv", encoding='utf-8')
amt = pd.DataFrame(csv)

# Extracting all categories from amenities 
amenities_mentioned = amt["Amenities"].str.split().explode().unique().tolist()
amenities_mentioned = [amenity.rstrip(",") for amenity in amenities_mentioned]

#remove irrelevant ones
remove = ["'HTTPSConnectionPool(host='nominatim.openstreetmap.org'", 'Max', 'retries', 'exceeded', 'with', 'url', 'Caused', 'by', 'Read', 'timed', 'out', 'read', "'NoneType'", 'object', 'has', 'no', 'attribute', "'latitude'", "read", "url", "Caused", "archive", "out", "HTTPSConnectionPool(host='nominatim.openstreetmap.org'"]
filtered_amenities= [amenity for amenity in amenities_mentioned if amenity not in remove]
filtered_amenities = [amenity for amenity in filtered_amenities if not any(error_word in amenity for error_word in ['Error', 'timeout', 'IncompleteRead', '/search?q='])]

# Remove non-alphabetic characters
filtered_amenities = [re.sub(r'[^a-zA-Z_]', '', amenity) for amenity in filtered_amenities]
filtered_amenities = [amenity for amenity in filtered_amenities if amenity]

#drop duplicates
amenities_mentioned = list(set(filtered_amenities))

# Print list of unique amenities
print(amenities_mentioned)

['toilets', 'ferry_terminal', 'motorcycle_parking', 'fountain', 'parking', 'taxi', 'studio', 'coworking_space', 'meeting_centre', 'compressed_air', 'restaurant', 'police', 'place_of_worship', 'balance_beam', 'animal_shelter', 'read', 'hospital', 'social_centre', 'fuel', 'veterinary', 'brothel', 'car_wash', 'photo_booth', 'pharmacy', 'recycling', 'mailboxes', 'food_court', 'vending_machine', 'accupuncture', 'public_bookcase', 'parking_entrance', 'playground', 'luggage_locker', 'bus_station', 'waste_basket', 'parking_space', 'pub', 'courthouse', 'dentist', 'Caused', 'social_facility', 'nursing_home', 'college', 'atm', 'lockers', 'freezone', 'locker', 'shelter', 'library', 'bicycle_repair_station', 'bar', 'theatre', 'fire_station', 'events_venue', 'health_centre', 'childcare', 'car_sharing', 'pedalo', 'bureau_de_change', 'bank', 'post_office', 'waste_disposal', 'townhall', 'school', 'bicycle_rental', 'music_school', 'clinic', 'music_studio', 'cooking_school', 'community_centre', 'training

In [3]:
excel = "3800_reviews_english.xlsx"

# Read Excel file into DataFrame
rating_amenities = pd.read_excel(excel)

# Drop specific columns from the original DataFrame
#keep place id, overall rating, individual rating
rating_amenities = rating_amenities.drop(["name", "review_text", "location_link", "reviews", "review_datetime_utc", "reviews_per_score_1", "reviews_per_score_2", "reviews_per_score_3", "reviews_per_score_4", "reviews_per_score_5", "query", "google_id", "reviews_link", "review_id", "review_pagination_id", "author_id", "review_text_english", "review_img_url", "review_img_url", "review_questions", "review_photo_ids", "owner_answer", "owner_answer_timestamp", "review_img_urls",  "owner_answer_timestamp_datetime_utc", "review_link", "review_timestamp", "review_likes", "reviews_id"], axis=1)

# Display new DataFrame
print(rating_amenities)

                         place_id  rating  review_rating
0     ChIJCwHQg38JxkcRJeVtM9mPCnM     3.2              1
1     ChIJCwHQg38JxkcRJeVtM9mPCnM     3.2              2
2     ChIJCwHQg38JxkcRJeVtM9mPCnM     3.2              5
3     ChIJCwHQg38JxkcRJeVtM9mPCnM     3.2              1
4     ChIJCwHQg38JxkcRJeVtM9mPCnM     3.2              5
...                           ...     ...            ...
3830  ChIJGU-a6R8XxkcR5cxWLxQYpgs     4.0              4
3831  ChIJGU-a6R8XxkcR5cxWLxQYpgs     4.0              5
3832  ChIJGU-a6R8XxkcR5cxWLxQYpgs     4.0              5
3833  ChIJGU-a6R8XxkcR5cxWLxQYpgs     4.0              4
3834  ChIJGU-a6R8XxkcR5cxWLxQYpgs     4.0              5

[3835 rows x 3 columns]


In [4]:
amenity_counts_dict = {amenity: [] for amenity in amenities_mentioned}

# Loop to count the amount of amenities per location
for amenities_str in amt["Amenities"]:
    for amenity in amenities_mentioned:
        count = amenities_str.count(amenity)
        
        amenity_counts_dict[amenity].append(count)

# Add the counts as new columns in amt
for amenity, counts in amenity_counts_dict.items():
    amt[f'{amenity}_counts'] = counts

amt["total_counts"] = amt[[f"{amenity}_counts" for amenity in amenities_mentioned]].sum(axis=1)

# Display
display(amt)



C:\Users\Usuario\AppData\Local\Temp\ipykernel_6060\2187263265.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  amt[f'{amenity}_counts'] = counts
C:\Users\Usuario\AppData\Local\Temp\ipykernel_6060\2187263265.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  amt[f'{amenity}_counts'] = counts
C:\Users\Usuario\AppData\Local\Temp\ipykernel_6060\2187263265.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all

,Address,place_id,Amenities,toilets_counts,ferry_terminal_counts,motorcycle_parking_counts,fountain_counts,parking_counts,taxi_counts,studio_counts,...,boat_rental_counts,ticket_validator_counts,port_counts,office_units_counts,nightclub_counts,casino_counts,cafe_counts,cannabis_counts,fast_food_counts,total_counts
0,"Julianaplein 1, Amsterdam",ChIJCwHQg38JxkcRJeVtM9mPCnM,"post_box, recycling, fast_food, restaurant, re...",2,0,0,0,10,0,0,...,1,0,0,0,0,0,2,0,3,110
1,"Joan Muyskenweg 22, 1096 CJ Amsterdam",ChIJxZHiajgLxkcRlOo3zmIibOs,"parking, post_box, parking, fuel, charging_sta...",0,0,1,0,3,0,0,...,0,0,0,0,0,0,0,0,0,20
2,"Nieuwe Leeuwarderweg, 1021 BZ Amsterdam",ChIJYyOpnFQIxkcRErWFhYVuAeQ,"fuel, post_box, fast_food, pharmacy, pub, pub,...",2,0,0,1,5,1,1,...,0,0,0,0,0,0,1,0,5,121
3,"Spoorslag 29, 1082 MM Amsterdam",ChIJ15K9igQKxkcR8A7le6H02hI,"cafe, parking, place_of_worship, cafe, parking...",1,0,1,0,15,2,0,...,0,0,0,0,0,0,16,0,5,149
4,"Julianaplein Oost, 1097 DN Amsterdam",ChIJPd2JhH8JxkcRQ8jSHUchci0,"post_box, recycling, fast_food, restaurant, re...",2,0,0,0,11,0,0,...,1,0,0,0,0,0,3,0,4,121
5,"Termini 23, Amsterdam",ChIJn0fdmPwJxkcRYe7a-6JHfpI,"townhall, healthcare, social_centre, dentist, ...",0,0,0,1,7,0,1,...,0,0,0,0,0,0,2,0,5,136
6,"Stationsplein 39, 1012 AB Amsterdam",ChIJf_z8xbcJxkcRCc-nQwIMAP8,"restaurant, pub, pub, pub, restaurant, bureau_...",6,1,0,0,25,2,0,...,0,0,0,0,1,2,32,2,62,463
7,"Cornelis Lelylaan 35, 1062 HD Amsterdam",ChIJ71V5XiPixUcRSKlH5OYy-iE,"recycling, recycling, recycling, post_box, res...",0,0,0,0,6,0,0,...,0,0,0,0,0,0,3,0,5,98
8,"Europaboulevard 4A, 1083 AD Amsterdam",ChIJLWMViCIKxkcRi1ThzDUCzCo,"theatre, conference_centre, parking, parking, ...",1,0,0,0,13,0,0,...,0,0,0,0,0,0,1,0,0,97
9,"Kattenburgerstraat 5, 1018 JA Amsterdam",ChIJ8exiz50JxkcRA0dHjOHV59g,"pub, post_box, restaurant, restaurant, restaur...",2,0,0,0,24,0,0,...,0,0,0,0,0,0,6,0,2,157


In [5]:
#add place ID
place_ID_df = pd.read_excel("final_input_addresses.xlsx")

# Merge DataFrames based on adress and safe place_id
merged_df = pd.merge(amt, place_ID_df, on = "Address", how = "left")

amt["place_id"] = place_ID_df.groupby("Address")["place_id"].transform("first")
display(amt)

,Address,place_id,Amenities,toilets_counts,ferry_terminal_counts,motorcycle_parking_counts,fountain_counts,parking_counts,taxi_counts,studio_counts,...,boat_rental_counts,ticket_validator_counts,port_counts,office_units_counts,nightclub_counts,casino_counts,cafe_counts,cannabis_counts,fast_food_counts,total_counts
0,"Julianaplein 1, Amsterdam",ChIJCwHQg38JxkcRJeVtM9mPCnM,"post_box, recycling, fast_food, restaurant, re...",2,0,0,0,10,0,0,...,1,0,0,0,0,0,2,0,3,110
1,"Joan Muyskenweg 22, 1096 CJ Amsterdam",ChIJxZHiajgLxkcRlOo3zmIibOs,"parking, post_box, parking, fuel, charging_sta...",0,0,1,0,3,0,0,...,0,0,0,0,0,0,0,0,0,20
2,"Nieuwe Leeuwarderweg, 1021 BZ Amsterdam",ChIJYyOpnFQIxkcRErWFhYVuAeQ,"fuel, post_box, fast_food, pharmacy, pub, pub,...",2,0,0,1,5,1,1,...,0,0,0,0,0,0,1,0,5,121
3,"Spoorslag 29, 1082 MM Amsterdam",ChIJ15K9igQKxkcR8A7le6H02hI,"cafe, parking, place_of_worship, cafe, parking...",1,0,1,0,15,2,0,...,0,0,0,0,0,0,16,0,5,149
4,"Julianaplein Oost, 1097 DN Amsterdam",ChIJPd2JhH8JxkcRQ8jSHUchci0,"post_box, recycling, fast_food, restaurant, re...",2,0,0,0,11,0,0,...,1,0,0,0,0,0,3,0,4,121
5,"Termini 23, Amsterdam",ChIJn0fdmPwJxkcRYe7a-6JHfpI,"townhall, healthcare, social_centre, dentist, ...",0,0,0,1,7,0,1,...,0,0,0,0,0,0,2,0,5,136
6,"Stationsplein 39, 1012 AB Amsterdam",ChIJf_z8xbcJxkcRCc-nQwIMAP8,"restaurant, pub, pub, pub, restaurant, bureau_...",6,1,0,0,25,2,0,...,0,0,0,0,1,2,32,2,62,463
7,"Cornelis Lelylaan 35, 1062 HD Amsterdam",ChIJ71V5XiPixUcRSKlH5OYy-iE,"recycling, recycling, recycling, post_box, res...",0,0,0,0,6,0,0,...,0,0,0,0,0,0,3,0,5,98
8,"Europaboulevard 4A, 1083 AD Amsterdam",ChIJLWMViCIKxkcRi1ThzDUCzCo,"theatre, conference_centre, parking, parking, ...",1,0,0,0,13,0,0,...,0,0,0,0,0,0,1,0,0,97
9,"Kattenburgerstraat 5, 1018 JA Amsterdam",ChIJ8exiz50JxkcRA0dHjOHV59g,"pub, post_box, restaurant, restaurant, restaur...",2,0,0,0,24,0,0,...,0,0,0,0,0,0,6,0,2,157


In [24]:
#merge the two to have ratings and amenities in one DataFrame
rating_and_count = pd.merge(amt, rating_amenities, on = "place_id", how = "left")

#turn index column into 
rating_and_count.reset_index(inplace = True)

#Display merged DataFrame
display(rating_and_count)

#Save to csv
rating_and_count.to_csv("rating_and_count.csv", index = False)
for col in rating_and_count.columns:
    print(col)
 
columns_to_drop = ['Address', 'index', 'place_id', 'Amenities', 'total_counts']
rating_and_count = rating_and_count.drop(columns=columns_to_drop)

all_columns = rating_and_count.columns.tolist()
print("List of all columns in the DataFrame:")
print(all_columns)


,index,Address,place_id,Amenities,toilets_counts,ferry_terminal_counts,motorcycle_parking_counts,fountain_counts,parking_counts,taxi_counts,...,port_counts,office_units_counts,nightclub_counts,casino_counts,cafe_counts,cannabis_counts,fast_food_counts,total_counts,rating,review_rating
0,0,"Julianaplein 1, Amsterdam",ChIJCwHQg38JxkcRJeVtM9mPCnM,"post_box, recycling, fast_food, restaurant, re...",2,0,0,0,10,0,...,0,0,0,0,2,0,3,110,3.2,1.0
1,1,"Julianaplein 1, Amsterdam",ChIJCwHQg38JxkcRJeVtM9mPCnM,"post_box, recycling, fast_food, restaurant, re...",2,0,0,0,10,0,...,0,0,0,0,2,0,3,110,3.2,2.0
2,2,"Julianaplein 1, Amsterdam",ChIJCwHQg38JxkcRJeVtM9mPCnM,"post_box, recycling, fast_food, restaurant, re...",2,0,0,0,10,0,...,0,0,0,0,2,0,3,110,3.2,5.0
3,3,"Julianaplein 1, Amsterdam",ChIJCwHQg38JxkcRJeVtM9mPCnM,"post_box, recycling, fast_food, restaurant, re...",2,0,0,0,10,0,...,0,0,0,0,2,0,3,110,3.2,1.0
4,4,"Julianaplein 1, Amsterdam",ChIJCwHQg38JxkcRJeVtM9mPCnM,"post_box, recycling, fast_food, restaurant, re...",2,0,0,0,10,0,...,0,0,0,0,2,0,3,110,3.2,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3839,3839,"Busplein 16, 1315 KR Almere",ChIJGU-a6R8XxkcR5cxWLxQYpgs,"parking, post_box, fast_food, fast_food, fast_...",1,0,0,0,6,0,...,0,0,0,1,5,0,20,95,4.0,4.0
3840,3840,"Busplein 16, 1315 KR Almere",ChIJGU-a6R8XxkcR5cxWLxQYpgs,"parking, post_box, fast_food, fast_food, fast_...",1,0,0,0,6,0,...,0,0,0,1,5,0,20,95,4.0,5.0
3841,3841,"Busplein 16, 1315 KR Almere",ChIJGU-a6R8XxkcR5cxWLxQYpgs,"parking, post_box, fast_food, fast_food, fast_...",1,0,0,0,6,0,...,0,0,0,1,5,0,20,95,4.0,5.0
3842,3842,"Busplein 16, 1315 KR Almere",ChIJGU-a6R8XxkcR5cxWLxQYpgs,"parking, post_box, fast_food, fast_food, fast_...",1,0,0,0,6,0,...,0,0,0,1,5,0,20,95,4.0,4.0


index
Address
place_id
Amenities
toilets_counts
ferry_terminal_counts
motorcycle_parking_counts
fountain_counts
parking_counts
taxi_counts
studio_counts
coworking_space_counts
meeting_centre_counts
compressed_air_counts
restaurant_counts
police_counts
place_of_worship_counts
balance_beam_counts
animal_shelter_counts
read_counts
hospital_counts
social_centre_counts
fuel_counts
veterinary_counts
brothel_counts
car_wash_counts
photo_booth_counts
pharmacy_counts
recycling_counts
mailboxes_counts
food_court_counts
vending_machine_counts
accupuncture_counts
public_bookcase_counts
parking_entrance_counts
playground_counts
luggage_locker_counts
bus_station_counts
waste_basket_counts
parking_space_counts
pub_counts
courthouse_counts
dentist_counts
Caused_counts
social_facility_counts
nursing_home_counts
college_counts
atm_counts
lockers_counts
freezone_counts
locker_counts
shelter_counts
library_counts
bicycle_repair_station_counts
bar_counts
theatre_counts
fire_station_counts
events_venue_coun

In [25]:
# Clean up column names to try and avoid "rating" not unique error
rating_and_count.head()
rating_and_count.info()
print(rating_and_count.dtypes)
print(rating_and_count.columns)


duplicate_columns = rating_and_count.columns[rating_and_count.columns.duplicated()]
print("Duplicate Columns:", duplicate_columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3844 entries, 0 to 3843
Columns: 112 entries, toilets_counts to review_rating
dtypes: float64(2), int64(110)
memory usage: 3.3 MB
toilets_counts                 int64
ferry_terminal_counts          int64
motorcycle_parking_counts      int64
fountain_counts                int64
parking_counts                 int64
                              ...   
cafe_counts                    int64
cannabis_counts                int64
fast_food_counts               int64
rating                       float64
review_rating                float64
Length: 112, dtype: object
Index(['toilets_counts', 'ferry_terminal_counts', 'motorcycle_parking_counts',
       'fountain_counts', 'parking_counts', 'taxi_counts', 'studio_counts',
       'coworking_space_counts', 'meeting_centre_counts',
       'compressed_air_counts',
       ...
       'ticket_validator_counts', 'port_counts', 'office_units_counts',
       'nightclub_counts', 'casino_counts', 'cafe_counts',

In [26]:
# Correlation between features and ratings

# List of features for correlation analysis
selected_features = all_columns
selected_features = [feature for feature in all_columns if feature != 'rating']

# Ensure unique elements in the list
selected_features = list(set(selected_features))

# Check if all selected features are present in the DataFrame
missing_features = [feature for feature in selected_features if feature not in rating_and_count.columns]

# If there are missing features, you can remove them from the selected_features list
if missing_features:
    print("The following features are missing in the DataFrame and will be removed:")
    print(missing_features)
    selected_features = [feature for feature in selected_features if feature in rating_and_count.columns]

# Calculate correlation matrix for selected features
correlation_matrix_selected = rating_and_count[selected_features + ['rating']].corr(numeric_only=True)

# Display the correlation matrix
print("Correlation Matrix:")
print(correlation_matrix_selected)

# Find features with the highest correlation with ratings

# Find features with the highest correlation with ratings
correlation_with_rating = correlation_matrix_selected[['rating']].abs()

# Sort the values in descending order
highest_correlation_selected = correlation_with_rating.sort_values(by='rating', ascending=False)

# Print the features with the highest correlation
print("\nFeatures with the highest correlation with ratings:")
print(highest_correlation_selected)

# Print the first 100 highest correlations with ratings
print("\nTop 100 Features with the Highest Correlation with Ratings:")
highest_correlation_selected_100 = highest_correlation_selected.head(100)
print(highest_correlation_selected_100)


selected_features_100 = highest_correlation_selected_100.index

# Convert the index to a list
selected_features100_list = list(selected_features_100)

# Print the list of features
print("Top 100 in list:")
print(selected_features100_list)

"""
Top 100 Features with the Highest Correlation with Ratings:
                            rating
rating                    1.000000
index                     0.436143
post_office,_counts       0.435711

Under "rating" is actually the correlation
"""

Correlation Matrix:
                        nursing_home_counts  vending_machine_counts  \
nursing_home_counts                1.000000                0.037572   
vending_machine_counts             0.037572                1.000000   
freezone_counts                   -0.002020               -0.022645   
veterinary_counts                 -0.012439                0.382604   
studio_counts                     -0.012185                0.204709   
...                                     ...                     ...   
dentist_counts                     0.262159                0.192058   
lockers_counts                    -0.016045               -0.073506   
restaurant_counts                 -0.034295                0.782628   
playground_counts                 -0.002858               -0.022570   
rating                            -0.118503                0.024184   

                        freezone_counts  veterinary_counts  studio_counts  \
nursing_home_counts           -0.002020          -

'\nTop 100 Features with the Highest Correlation with Ratings:\n                            rating\nrating                    1.000000\nindex                     0.436143\npost_office,_counts       0.435711\n\nUnder "rating" is actually the correlation\n'

In [28]:
"""
WORK IN PROGRESS: 

Make correlation score between all of them (all possible combinations). 
When highly correlated, drop one of the two. 
Take code from Vanessa?  

Matrix: all possible features x all possible features and correlation scores for all of them

->

import itertools
import pandas as pd

# Assuming 'Amenities' column contains strings with comma-separated amenity names
rating_and_count['Amenities'] = rating_and_count['Amenities'].astype(str)
rating_and_count['Amenities'] = rating_and_count['Amenities'].str.split(',')

# Create a list of all amenities
amenities = list(set(itertools.chain.from_iterable(rating_and_count['Amenities'])))

# Create all possible combinations of amenities
combinations = list(itertools.combinations(amenities, 2))

# Create an empty DataFrame to store correlation scores
correlation_scores = pd.DataFrame(index=amenities, columns=amenities)

# Calculate correlation scores for all combinations
for amenity1, amenity2 in combinations:
    correlation_score = rating_and_count[amenity1].corr(rating_and_count[amenity2])
    correlation_scores.loc[amenity1, amenity2] = correlation_score
    correlation_scores.loc[amenity2, amenity1] = correlation_score

# Identify highly correlated features
highly_correlated_features = set()
for amenity in amenities:
    correlated_with_amenity = correlation_scores[amenity][abs(correlation_scores[amenity]) > 0.8].index.tolist()
    highly_correlated_features.update(correlated_with_amenity)

# Display highly correlated features
print("\nHighly Correlated Features to be Dropped:")
print(highly_correlated_features)

# Drop the highly correlated features from the dataframe
rating_and_count_filtered = rating_and_count.drop(columns=highly_correlated_features)

# Display the filtered dataframe
print("\nFiltered DataFrame after dropping highly correlated features:")
print(rating_and_count_filtered)


ORIGINAL WORKING CODE BELLOW: 
"""

# Keep only the selected features in the DataFrame
df_regr = rating_and_count[selected_features100_list + ['review_rating']]

# # Calculate correlation matrix for all columns
correlation_matrix = df_regr.corr()

# Identify highly correlated features
highly_correlated_features = set()
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > 0.8:
            colname_i = correlation_matrix.columns[i]
            colname_j = correlation_matrix.columns[j]

            # Drop the first feature in the pair
            highly_correlated_features.add(colname_i)

# Display highly correlated features to be dropped
print("\nHighly Correlated Features to be Dropped:")
print(highly_correlated_features)

# Drop the highly correlated features from the dataframe
df_regr_filtered = df_regr.drop(columns=highly_correlated_features)


# Drop the highly correlated features from the dataframe
rating_and_count_filtered = rating_and_count.drop(columns=highly_correlated_features)

# Display the filtered dataframe
print("\nFiltered DataFrame after dropping highly correlated features:")
print(rating_and_count_filtered)

#Drops 62 columns, 66 amenities remaining



Highly Correlated Features to be Dropped:
{'bank_counts', 'vending_machine_counts', 'arts_centre_counts', 'accupuncture_counts', 'fire_station_counts', 'community_centre_counts', 'veterinary_counts', 'recycling_counts', 'health_centre_counts', 'warehouse_counts', 'animal_shelter_counts', 'photo_booth_counts', 'karaoke_box_counts', 'parking_counts', 'townhall_counts', 'theatre_counts', 'cannabis_counts', 'taxi_counts', 'healthcare_counts', 'port_counts', 'ice_cream_counts', 'atm_counts', 'out_counts', 'url_counts', 'clinic_counts', 'bicycle_repair_station_counts', 'university_counts', 'nightclub_counts', 'car_wash_counts', 'music_studio_counts', 'luggage_locker_counts', 'toilets_counts', 'locker_counts', 'meeting_centre_counts', 'ticket_validator_counts', 'post_box_counts', 'charging_station_counts', 'parcel_locker_counts', 'review_rating', 'bureau_de_change_counts', 'lockers_counts', 'fast_food_counts', 'cafe_counts', 'Caused_counts', 'restaurant_counts', 'playground_counts'}

Filtere